In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import torch.utils.data as td


In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

#         self.x_input = tf.placeholder(tf.float32, shape = [None, 784])
#         self.y_input = tf.placeholder(tf.int64, shape = [None])
#         self.x_image = tf.reshape(self.x_input, [-1, 28, 28, 1])

        # first convolutional layer
        # In channels, out channels, square kernel size, stride padding
        self.conv1 = nn.Conv2d(1,32,5,1,1)

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        # second convolutional layer
        self.conv2 = nn.Conv2d(32,64,5,1,1)

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        # first fully connected layer
        self.fc1 = nn.Linear(7*7*64, 1024)

        # output layer
        self.fc2 = nn.Linear(1024, 10)
        
#         y_xent = tf.nn.sparse_softmax_cross_entropy_with_logits(
#             labels=self.y_input, logits=self.pre_softmax)
        self.xent = nn.CrossEntropyLoss()
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 7*7*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#         self.y_pred = tf.argmax(self.pre_softmax, 1)
#         correct_prediction = tf.equal(self.y_pred, self.y_input)
#         self.num_correct = tf.reduce_sum(tf.cast(correct_prediction, tf.int64))
#         self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
class LinfPGDAttack:
    def __init__(self, model, cost, epsilon, k, a, random_start, loss_func):
        """Attack parameter initialization. The attack performs k steps of
           size a, while always staying within epsilon from the initial
           point."""
        self.model = model
        self.epsilon = epsilon
        self.k = k
        self.a = a
        self.rand = random_start

        if loss_func == 'xent':
            loss = model.xent
        elif loss_func == 'cw':
            label_mask = tf.one_hot(model.y_input,
                                  10,
                                  on_value=1.0,
                                  off_value=0.0,
                                  dtype=tf.float32)
            correct_logit = tf.reduce_sum(label_mask * model.pre_softmax, axis=1)
            wrong_logit = tf.reduce_max((1-label_mask) * model.pre_softmax, axis=1)
            loss = -tf.nn.relu(correct_logit - wrong_logit + 50)
        else:
            print('Unknown loss function. Defaulting to cross-entropy')
            loss = model.xent

        self.grad = tf.gradients(loss, model.x_input)[0]

    def perturb(self, x_nat, y, sess):
        """Given a set of examples (x_nat, y), returns a set of adversarial
           examples within epsilon of x_nat in l_infinity norm."""
        if self.rand:
            x = x_nat + np.random.uniform(-self.epsilon, self.epsilon, x_nat.shape)
        else:
            x = np.copy(x_nat)

        for i in range(self.k):
            grad = sess.run(self.grad, feed_dict={self.model.x_input: x,
                                                  self.model.y_input: y})

            x += self.a * np.sign(grad)

            x = np.clip(x, x_nat - self.epsilon, x_nat + self.epsilon) 
            x = np.clip(x, 0, 1) # ensure valid pixel range

        return x